# Plagiat Checker Gruppe A #

In [20]:
#Importe
import re  #Importiert das re Modul für reguläre Ausdrücke
import hashlib  
import numpy as np  

#Anpassung der Datei
def textanpassung(text):
    #Ersetzt Sonderzeichen durch "nichts" und konvertiert den Text in Kleinbuchstaben
    text = re.sub(r'[^\w\s]', '', text).lower()
    return text

#Erstellung von Shingles der Länge k
def shinglesErstellen(text, k=3):
    woerter = text.split()  # Zerlegt den Text in Wörter
    for i in range(len(woerter) - k + 1):
        yield ' '.join(woerter[i:i + k])

#Hashing mit der Minhash Methode
def minhash(shingles, permutationen=256):
    signaturen = np.full(permutationen, float('inf'))
    #Erstellung der Signaturen
    for shingle in shingles:
        for i in range(permutationen):
            #Hashing mit SHA256 aus hashlib library
            hashWert = int(hashlib.sha256(f'{shingle}{i}'.encode()).hexdigest(), 16) % (2**32 - 1) 
            #Speichert den kleinsten Hash-Wert für jede Permutation.
            signaturen[i] = min(signaturen[i], hashWert)
    return signaturen

#Definition der Jaccard Similarity
def jaccardSimilarity(set1, set2):
    #Vereinigungsmenge der Hash-Werte im Verhältnis zum Vocabulary
    return np.sum(set1 == set2) / len(set1)

#Aufruf der Datei
file_path = '/Users/furkan/Fortgeschrittene-Algorithmen-1/Aufgabe 3/articles_test.txt' 
artikelMinhashed = {}  #Für Speicherung der MinHash-Signaturen der Artikel.
#Öffnen und Enconding der Datei
with open(file_path, 'r', encoding='utf-8') as file:
    daten = file.readlines()

#Anpassung der Datei.
for zeilen in daten:
    artikel = zeilen.split(' ', 1)  #Trennt die Artikel-ID vom Text.
    if len(artikel) == 2:
        artikel_id, artikel_text = artikel[0], artikel[1]
        textAngepasst = textanpassung(artikel_text)
        shingles = set(shinglesErstellen(textAngepasst))
        minhashSignatur = minhash(shingles)
        artikelMinhashed[artikel_id] = minhashSignatur

#Suche nach ähnlichen Artikel-Paaren und berechne die Jaccard-Ähnlichkeit
plagiatKandidaten = []  # Liste zur Speicherung von Plagiat-Kandidaten
artikel_ID = list(artikelMinhashed.keys())  # Liste der Artikel-IDs

#Durchläuft jedes mögliche Paar von Artikeln
plagiate_gefunden = False
for i in range(len(artikel_ID)):
    for j in range(i + 1, len(artikel_ID)):
        id1, id2 = artikel_ID[i], artikel_ID[j]
        # erechnet die Jaccard-Ähnlichkeit der MinHash-Signaturen
        similarity = jaccardSimilarity(artikelMinhashed[id1], artikelMinhashed[id2])
        if similarity > 0.8:  # Überprüft, ob die Ähnlichkeit über dem Schwellenwert von 0.8 (80%) liegt. 
            #Fügt das Paar und seine Ähnlichkeit zur Liste hinzu
            plagiatKandidaten.append((id1, id2, similarity))
            plagiate_gefunden = True

#Ausgabe der Ergebnisse
if plagiate_gefunden:
    for paare in plagiatKandidaten:
        print(f"Artikel {paare[0]} und {paare[1]} ähneln sich mit einer Ähnlichkeit von {paare[2]*100:.2f}%.")

    print("Anzahl der Plagiate:",len(plagiatKandidaten))
else:
    print("Es wurden keine Plagiate gefunden")





Es wurden keine Plagiate gefunden 


# Quellen # 
- Reguläre Ausdrücke(Suche) https://cheatography.com/davechild/cheat-sheets/regular-expressions/
- Modul Script
- https://docs.python.org/3/library/hashlib.html für Algorithmus SHA256
- https://en.wikipedia.org/wiki/MinHash
- https://de.wikipedia.org/wiki/Jaccard-Koeffizient
- https://www.codemotion.com/magazine/backend/fast-document-similarity-in-python-minhashlsh/